In [29]:
import pandas as pd
import numpy as np
import glob, os
import time
import pickle
import matplotlib.pyplot as plt
import math
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, TimeDistributed
from keras.layers import LSTM, GRU
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from keras.optimizers import SGD

#np.seed(7)

with open('finalset/preprocess_48_hours.pickle', 'rb') as handle:
    [X,  Y] = pickle.load(handle)
df_murda = Y['LABEL']==1
df_murda = df_murda.as_matrix()
Y.drop(['SUBJECT_ID'],1, inplace=True)
Y = np.array(Y)
print(len(X),len(Y))
print(X.shape)
print(Y.shape)
X_murda = X[df_murda]
X_murda = np.repeat(X_murda,1,axis=0)
X = np.concatenate((X,X_murda),axis=0)
print(X.shape)
Y_murda = Y[df_murda]
Y_murda = np.repeat(Y_murda,1,axis=0)
Y = np.concatenate((Y,Y_murda),axis=0)
Y.shape

6587 6587
(6587, 49, 37)
(6587, 1)
(9048, 49, 37)


/home/yoda/.local/lib/python3.5/site-packages/ipykernel_launcher.py:20: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


(9048, 1)

In [30]:
X_new = X.reshape(X.shape[0],1,X.shape[1],X.shape[2])
Y_new = Y.reshape(Y.shape[0],1,Y.shape[1])
print(Y_new.shape)
X_new = np.insert(X_new, 1, Y_new, axis=1)
np.random.shuffle(X_new)
X_new.shape

(9048, 1, 1)


(9048, 2, 49, 37)

In [31]:
X = X_new[:,0]

X.shape

(9048, 49, 37)

In [32]:
Y = X_new[:,1]
Y = Y[:,0]
Y = Y[:,0]
Y = Y.reshape(Y.shape[0],1)
Y.shape

(9048, 1)

In [33]:
X = (X-np.mean(X))/(np.std(X))
X.shape

(9048, 49, 37)

In [34]:
from sklearn.model_selection import StratifiedShuffleSplit
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.3, random_state=2)
sss.get_n_splits(X, Y)

for train_index, test_index in sss.split(X, Y):
    #print("TRAIN:", train_index, "TEST:", test_index)
    train_X, test1_X = X[train_index], X[test_index]
    train_Y, test1_Y = Y[train_index], Y[test_index]

In [35]:
sss1 = StratifiedShuffleSplit(n_splits=1, test_size=0.5, random_state=0)
sss1.get_n_splits(test1_X, test1_Y)

for test_index, val_index in sss1.split(test1_X, test1_Y):
    #print("TRAIN:", train_index, "TEST:", test_index)
    test_X, val_X = test1_X[test_index], test1_X[val_index]
    test_Y, val_Y = test1_Y[test_index], test1_Y[val_index]

In [36]:
print(train_X.shape, test_X.shape, train_Y.shape, test_Y.shape, val_X.shape, val_Y.shape)

(6333, 49, 37) (1357, 49, 37) (6333, 1) (1357, 1) (1358, 49, 37) (1358, 1)


In [37]:
print(len(train_X), len(test_X))
print(len(train_Y), len(test_Y))

6333 1357
6333 1357


In [38]:
# lstm_size = []
# drpout = []
# dense1 = []
# dense2 = []
# dense3 = []

# # create and fit the LSTM network
# model = Sequential()
# model.add(LSTM(27, return_sequences = True, input_dim = 37, input_length = 49))
# model.add(Dropout(0.5))
# print model.output_shape
# #model.add(LSTM(14, return_sequences = True))
# #model.add(Dropout(0.5))
# model.add(Flatten())
# print model.output_shape
# model.add(Dense(100, init='glorot_uniform', activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(10, init='glorot_uniform', activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(5, init='glorot_uniform', activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(1, init='glorot_uniform', activation='sigmoid'))
# model.compile(loss='binary_crossentropy', optimizer='RMSprop', metrics=['accuracy'])
# model.fit(train_X, train_Y, nb_epoch=100, batch_size=40, verbose=2)
# val_predict = model.predict(val_X,  batch_size=40)
# from sklearn.metrics import roc_auc_score
# roc_auc_score(val_Y, val_predict)

In [ ]:
from sklearn.metrics import mean_squared_error
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import StratifiedKFold
import pandas as pd;
import numpy as np;
from sklearn import model_selection

best_params = None
best_score = 1000
auc = None
model_final = None
def score(params):
    global best_score
    global best_params
    global auc
    global model_final
    print("Training with params : ")
    print(params)
    lstm_size = params['lstm_size']
    attention = params['attention']
    dropout = params['dropout']
    dense1 = params['dense1']
    dense2 = params['dense2']
    dense3 = params['dense3']

    
    model = Sequential()
    model.add(LSTM(lstm_size, return_sequences = True, input_dim = 37, input_length = 49))
    model.add(TimeDistributed(Dense(attention, activation='tanh')))
    #model.add(Dropout(dropout))
    #print model.output_shape
    #model.add(LSTM(14, return_sequences = True))
    #model.add(Dropout(0.5))
    model.add(Flatten())
    #print model.output_shape
    model.add(Dense(dense1, init='glorot_uniform', activation='relu'))
    model.add(Dropout(dropout))
    model.add(Dense(dense2, init='glorot_uniform', activation='relu'))
    model.add(Dropout(dropout))
    model.add(Dense(dense3, init='glorot_uniform', activation='relu'))
    model.add(Dropout(dropout))
    model.add(Dense(1, init='glorot_uniform', activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='RMSprop', metrics=['accuracy'])
    model.fit(train_X, train_Y, nb_epoch=1, batch_size=40, verbose=0)
    val_predict = model.predict(val_X,  batch_size=40)
    test_predict=model.predict(test_X,batch_size=40)
    from sklearn.metrics import roc_auc_score, f1_score,precision_recall_fscore_support
    from sklearn.metrics import confusion_matrix
    score = 1.0 - roc_auc_score(val_Y, val_predict)
    f_score=f1_score(test_Y,test_predict.round())
    prfs=precision_recall_fscore_support(test_Y,test_predict.round())
    f_score_micro=f1_score(test_Y,test_predict.round(),average='micro')
    f_score_macro=f1_score(test_Y,test_predict.round(),average='macro')
    cf=confusion_matrix(test_Y,test_predict.round())
    #scores = model_selection.cross_val_score(model, X, y, scoring='roc_auc', cv=sss, n_jobs=-1, verbose=0)
    #score = 1.0-scores.mean()
    if best_score > score:
        best_score = score
        best_params = params
        auc = 1.0 - score
        model_final = model
    print("AUC : "+str(1.0 - score)+",  Score/Loss : "+str(score)+", F1_Score_average: "+str(f_score)+", Precision, Recall, F1_score, Support: "+str(prfs))
    print("F1_score_micro: "+str(f_score_micro))
    print("F1_score_macro: "+str(f_score_macro))
    print("Confusion Matrix")
    print(cf)
    print("END\n\n")
    return {'loss': score, 'status': STATUS_OK}

def optimize(trials):
    space = {
    'lstm_size': hp.choice('lstm_size', np.arange(15, 35, 5, dtype=int)),
    'attention': hp.choice('attention',[10, 20, 30, 40]),  
    'dropout': hp.choice('dropout',[0.5, 0.3]),
    'dense1': hp.choice('dense1', [70, 80, 90, 100, 125]),
    'dense2' : hp.choice('dense2', np.arange(8, 20, 3, dtype=int)),
    "dense3": hp.choice('dense3', np.arange(5, 8, 1, dtype=int))
    }
    best = fmin(score, space, algo=tpe.suggest, trials=trials, max_evals=40)
    #print "optimize best", best
    return best



# from sklearn import datasets
# iris = datasets.load_iris()
# iris_X = iris.data
# iris_y = iris.target
# #print set(iris_y)

# X, y = iris_X,iris_y
# y[y[:] == 2] = 1
# y[y[:] == 3] = 2

# #print set(y)
# #print "Splitting data into train and valid ...\n\n"
# sss = StratifiedKFold(y, n_folds=3,shuffle=True, random_state=42)

# #Trials object where the history of search will be stored
trials = Trials()
best = optimize(trials)
#print "best", best
print("best_score : ", best_score)
print("best_params\n\n", best_params)
print("\n\nauc : ", auc)

Training with params : 
{'attention': 10, 'dropout': 0.3, 'dense3': 6, 'dense1': 90, 'lstm_size': 15, 'dense2': 14}


/home/yoda/.local/lib/python3.5/site-packages/ipykernel_launcher.py:29: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
/home/yoda/.local/lib/python3.5/site-packages/ipykernel_launcher.py:29: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(15, return_sequences=True, input_shape=(49, 37))`
/home/yoda/.local/lib/python3.5/site-packages/ipykernel_launcher.py:37: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(90, activation="relu", kernel_initializer="glorot_uniform")`
/home/yoda/.local/lib/python3.5/site-packages/ipykernel_launcher.py:39: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(14, activation="relu", kernel_initializer="glorot_uniform")`
/home/yoda/.local/lib/python3.5/site-packages/ipykernel_launcher.py:41: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(6, activation="relu", kernel_initializer="glorot_uniform")`
/home/yoda/.local/lib/python3

AUC : 0.5625392127072126,  Score/Loss : 0.43746078729278737, F1_Score_average: 0.6958230958230958, Precision, Recall, F1_score, Support: (array([0.5      , 0.5458751]), array([0.04846527, 0.95934959]), array([0.08836524, 0.6958231 ]), array([619, 738]))
F1_score_micro: 0.5438467207074429
F1_score_macro: 0.392094169413757
Confusion Matrix
[[ 30 589]
 [ 30 708]]
END


Training with params : 
{'attention': 40, 'dropout': 0.3, 'dense3': 7, 'dense1': 90, 'lstm_size': 25, 'dense2': 17}


/home/yoda/.local/lib/python3.5/site-packages/ipykernel_launcher.py:29: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
/home/yoda/.local/lib/python3.5/site-packages/ipykernel_launcher.py:29: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(25, return_sequences=True, input_shape=(49, 37))`
/home/yoda/.local/lib/python3.5/site-packages/ipykernel_launcher.py:37: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(90, activation="relu", kernel_initializer="glorot_uniform")`
/home/yoda/.local/lib/python3.5/site-packages/ipykernel_launcher.py:39: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(17, activation="relu", kernel_initializer="glorot_uniform")`
/home/yoda/.local/lib/python3.5/site-packages/ipykernel_launcher.py:41: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(7, activation="relu", kernel_initializer="glorot_uniform")`
/home/yoda/.local/lib/python3

AUC : 0.5532013964642435,  Score/Loss : 0.4467986035357565, F1_Score_average: 0.669050051072523, Precision, Recall, F1_score, Support: (array([0.39416058, 0.53688525]), array([0.08723748, 0.88753388]), array([0.14285714, 0.66905005]), array([619, 738]))
F1_score_micro: 0.522476050110538
F1_score_macro: 0.40595359696483296
Confusion Matrix
[[ 54 565]
 [ 83 655]]
END


Training with params : 
{'attention': 40, 'dropout': 0.3, 'dense3': 5, 'dense1': 100, 'lstm_size': 30, 'dense2': 11}


...

/home/yoda/.local/lib/python3.5/site-packages/ipykernel_launcher.py:29: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
/home/yoda/.local/lib/python3.5/site-packages/ipykernel_launcher.py:29: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(20, return_sequences=True, input_shape=(49, 37))`
/home/yoda/.local/lib/python3.5/site-packages/ipykernel_launcher.py:37: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(70, activation="relu", kernel_initializer="glorot_uniform")`
/home/yoda/.local/lib/python3.5/site-packages/ipykernel_launcher.py:39: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(17, activation="relu", kernel_initializer="glorot_uniform")`
/home/yoda/.local/lib/python3.5/site-packages/ipykernel_launcher.py:41: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(7, activation="relu", kernel_initializer="glorot_uniform")`
/home/yoda/.local/lib/python3

AUC : 0.5354799853970238,  Score/Loss : 0.4645200146029762, F1_Score_average: 0.7045346062052505, Precision, Recall, F1_score, Support: (array([0.        , 0.54384672]), array([0., 1.]), array([0.        , 0.70453461]), array([619, 738]))
F1_score_micro: 0.5438467207074429
F1_score_macro: 0.3522673031026253
Confusion Matrix
[[  0 619]
 [  0 738]]
END


Training with params : 
{'attention': 20, 'dropout': 0.3, 'dense3': 7, 'dense1': 125, 'lstm_size': 20, 'dense2': 8}


/home/yoda/.local/lib/python3.5/site-packages/ipykernel_launcher.py:41: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(7, activation="relu", kernel_initializer="glorot_uniform")`
/home/yoda/.local/lib/python3.5/site-packages/ipykernel_launcher.py:43: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, activation="sigmoid", kernel_initializer="glorot_uniform")`
/home/yoda/.local/lib/python3.5/site-packages/ipykernel_launcher.py:45: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


AUC : 0.521535017630689,  Score/Loss : 0.478464982369311, F1_Score_average: 0.6624605678233438, Precision, Recall, F1_score, Support: (array([0.44041451, 0.54123711]), array([0.13731826, 0.85365854]), array([0.20935961, 0.66246057]), array([619, 738]))
F1_score_micro: 0.52689756816507
F1_score_macro: 0.43591008686733695
Confusion Matrix
[[ 85 534]
 [108 630]]
END


Training with params : 
{'attention': 20, 'dropout': 0.3, 'dense3': 7, 'dense1': 90, 'lstm_size': 20, 'dense2': 17}


/home/yoda/.local/lib/python3.5/site-packages/ipykernel_launcher.py:29: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
/home/yoda/.local/lib/python3.5/site-packages/ipykernel_launcher.py:29: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(20, return_sequences=True, input_shape=(49, 37))`
/home/yoda/.local/lib/python3.5/site-packages/ipykernel_launcher.py:37: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(90, activation="relu", kernel_initializer="glorot_uniform")`
/home/yoda/.local/lib/python3.5/site-packages/ipykernel_launcher.py:39: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(17, activation="relu", kernel_initializer="glorot_uniform")`
/home/yoda/.local/lib/python3.5/site-packages/ipykernel_launcher.py:41: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(7, activation="relu", kernel_initializer="glorot_uniform")`
/home/yoda/.local/lib/python3

In [ ]:
test_predict = model_final.predict(test_X,  batch_size=40)
#print(test_predict.shape)
score = model_final.evaluate(test_X, test_Y, verbose=0)
print(score)
from sklearn.metrics import roc_auc_score
print(roc_auc_score(test_Y, test_predict))

In [15]:
from keras.models import load_model
model.save('model.h5')

In [27]:
model = Sequential()
model.add(LSTM(20, return_sequences = 20, input_dim = 37, input_length = 49))
model.add(Dropout(0.3))
#print model.output_shape
#model.add(LSTM(14, return_sequences = True))
#model.add(Dropout(0.5))
model.add(Flatten())
#print model.output_shape
model.add(Dense(125, init='glorot_uniform', activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(11, init='glorot_uniform', activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(7, init='glorot_uniform', activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1, init='glorot_uniform', activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='RMSprop', metrics=['accuracy'])
model.fit(train_X, train_Y, nb_epoch=10, batch_size=40, verbose=1)
val_predict = model.predict(val_X,  batch_size=40)
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
test_predict = model.predict(test_X,  batch_size=40)

print(roc_auc_score(val_Y, val_predict))
print(roc_auc_score(test_Y, test_predict))

/home/yoda/.local/lib/python3.5/site-packages/ipykernel_launcher.py:2: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
  
/home/yoda/.local/lib/python3.5/site-packages/ipykernel_launcher.py:2: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(20, return_sequences=20, input_shape=(49, 37))`
  
/home/yoda/.local/lib/python3.5/site-packages/ipykernel_launcher.py:9: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(125, activation="relu", kernel_initializer="glorot_uniform")`
  if __name__ == '__main__':
/home/yoda/.local/lib/python3.5/site-packages/ipykernel_launcher.py:11: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(11, activation="relu", kernel_initializer="glorot_uniform")`
  # This is added back by InteractiveShellApp.init_path()
/home/yoda/.local/lib/python3.5/site-packages/ipykernel_launcher.py:13: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(

Epoch 1/10
5663/5663 [==============================] - 16s 3ms/step - loss: 0.6940 - acc: 0.5352
Epoch 2/10
5663/5663 [==============================] - 4s 780us/step - loss: 0.6846 - acc: 0.5400
Epoch 3/10
5663/5663 [==============================] - 4s 762us/step - loss: 0.6809 - acc: 0.5403
Epoch 4/10
5663/5663 [==============================] - 5s 851us/step - loss: 0.6842 - acc: 0.5430
Epoch 5/10
5663/5663 [==============================] - 5s 808us/step - loss: 0.6848 - acc: 0.5411
Epoch 6/10
5663/5663 [==============================] - 4s 774us/step - loss: 0.6814 - acc: 0.5476
Epoch 7/10
5663/5663 [==============================] - 4s 771us/step - loss: 0.6845 - acc: 0.5458
Epoch 8/10
5663/5663 [==============================] - 5s 866us/step - loss: 0.6811 - acc: 0.5418
Epoch 9/10
5663/5663 [==============================] - 5s 798us/step - loss: 0.6838 - acc: 0.5483
Epoch 10/10
5663/5663 [==============================] - 4s 772us/step - loss: 0.6818 - acc: 0.5524
0.55636920

In [28]:
test_predict = model.predict(test_X,  batch_size=40)
#print(test_predict.shape)
score = model.evaluate(test_X, test_Y, verbose=0)
print(score)
from sklearn.metrics import roc_auc_score
print(roc_auc_score(test_Y, test_predict))

[0.6829080685752228, 0.5560131796698594]
0.5509963433173097


In [11]:
model = Sequential()
model.add(LSTM(20, return_sequences = True, input_dim = 37, input_length = 49))
model.add(Dropout(0.5))
#print model.output_shape
#model.add(LSTM(14, return_sequences = True))
#model.add(Dropout(0.5))
model.add(Flatten())
#print model.output_shape
model.add(Dense(125, init='glorot_uniform', activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(11, init='glorot_uniform', activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(7, init='glorot_uniform', activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, init='glorot_uniform', activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='RMSprop', metrics=['accuracy'])
model.fit(train_X, train_Y, nb_epoch=100, batch_size=40, verbose=2)
val_predict = model.predict(val_X,  batch_size=40)
test_predict = model.predict(test_X,  batch_size=1)
# print test_predict
# test_out=model_final.predict_classes(test_X,batch_size=40)
# val_out=model.predict_classes(val_X,  batch_size=40)
from sklearn.metrics import roc_auc_score,f1_score,precision_recall_fscore_support
print "Accuracy on validation dataset:"
print roc_auc_score(val_Y,val_predict)
print "Accuracy on test dataset:"
print roc_auc_score(test_Y,test_predict)
for i in range(1,len(test_predict)):
    print "Patient "+str(i)+ " critical illness probability: "+str(test_predict[i][0])
# print precision_recall_fscore_support(test_Y,test_predict.round())
# print f1_score(test_Y,test_predict.round(),average='macro')
# print f1_score(test_Y,test_predict.round(),average='micro')

/home/iiitm/.local/lib/python2.7/site-packages/ipykernel_launcher.py:2: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
  
/home/iiitm/.local/lib/python2.7/site-packages/ipykernel_launcher.py:2: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(20, return_sequences=True, input_shape=(49, 37))`
  
/home/iiitm/.local/lib/python2.7/site-packages/ipykernel_launcher.py:9: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(125, activation="relu", kernel_initializer="glorot_uniform")`
  if __name__ == '__main__':
/home/iiitm/.local/lib/python2.7/site-packages/ipykernel_launcher.py:11: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(11, activation="relu", kernel_initializer="glorot_uniform")`
  # This is added back by InteractiveShellApp.init_path()
/home/iiitm/.local/lib/python2.7/site-packages/ipykernel_launcher.py:13: UserWarning: Update your `Dense` call to the Keras 2 API: 

Epoch 1/100
0s - loss: 0.7007 - acc: 0.5000
Epoch 2/100
0s - loss: 0.6520 - acc: 0.5000
Epoch 3/100
0s - loss: 0.6647 - acc: 0.6364
Epoch 4/100
0s - loss: 0.5765 - acc: 0.6364
Epoch 5/100
0s - loss: 0.7702 - acc: 0.6364
Epoch 6/100
0s - loss: 0.6653 - acc: 0.6818
Epoch 7/100
0s - loss: 0.7310 - acc: 0.6364
Epoch 8/100
0s - loss: 0.6079 - acc: 0.6364
Epoch 9/100
0s - loss: 0.6622 - acc: 0.6818
Epoch 10/100
0s - loss: 0.5511 - acc: 0.7727
Epoch 11/100
0s - loss: 0.6399 - acc: 0.7273
Epoch 12/100
0s - loss: 0.6586 - acc: 0.5455
Epoch 13/100
0s - loss: 0.6123 - acc: 0.6818
Epoch 14/100
0s - loss: 0.6352 - acc: 0.6364
Epoch 15/100
0s - loss: 0.5678 - acc: 0.6364
Epoch 16/100
0s - loss: 0.6097 - acc: 0.7273
Epoch 17/100
0s - loss: 0.6475 - acc: 0.7273
Epoch 18/100
0s - loss: 0.6234 - acc: 0.6818
Epoch 19/100
0s - loss: 0.6129 - acc: 0.6364
Epoch 20/100
0s - loss: 0.5391 - acc: 0.7273
Epoch 21/100
0s - loss: 0.5309 - acc: 0.7273
Epoch 22/100
0s - loss: 0.5764 - acc: 0.7273
Epoch 23/100
0s - l